## Compressed Model Performance Benchmarking Using GuideLLM

This notebook focuses on evaluating the system-level performance of the **compressed model**. The results are used to understand how compression affects latency, throughput, and scalability compared to the uncompressed baseline.

**Goal**

Assess the performance efficiency of the compressed model and quantify the gains or trade-offs introduced by compression under realistic serving conditions.

**GuideLLM Overview**

GuideLLM is an open-source benchmarking tool used to measure the performance of large language models deployed with **vLLM**. It captures detailed system and inference-level metrics, including:

- *Token throughput*
- *Latency metrics*
  - Time to First Token (TTFT)
  - Inter-Token Latency (ITL)
  - End-to-end request latency
- *Concurrency behavior*
- *Request-level diagnostics*

**Prerequisites**

To run performance benchmarking using GuideLLM, we first need to start a vLLM server to host the base model.

More details on system level performance benchmarking and GuideLLM are provided in [System_Level_Performance_Benchmarking.md](../docs/System_Level_Performance_Benchmarking.md)

### Install Depoendencies

In [ ]:
# uncomment the following lines to install dependencies if dependencies were not installed in 02_Base_Performance_Benchmarking/Base.ipynb
# !pip install .

In [ ]:
import os

from guidellm.benchmark import GenerativeBenchmarksReport
from utils import generate, stream

### Launch an Inference Server (vLLM) for the compressed Model

Set up a vLLM inference server to host the compressed model and expose an OpenAI-compatible API endpoint. This server is required so that GuideLLM can benchmark system-level performance like throughput, latency, and time-to-first-token. The performance benchmarks of the base and compressed models will be used later on to draw comparisons.

The compressed model will be accessible via an API for performance evaluation.

**Resources used** : 46GB L40S GPU x 1

More details on vLLM are provided in [Model_Serving_vLLM.md](../docs/Vllm_Server_README.md)

####  Set up Environment Variables

In [ ]:
# set the logging level for vLLM inference
os.environ["VLLM_LOGGING_LEVEL"] = "DEBUG"

**Before starting this notebook, use `nvidia-smi` and then `kill -9 <pid>` to kill any running processes that might be consuming GPU memory.**

#### vLLM config for single node

We will be using the configuration for a single-node, single-GPU set up to launch a vLLM server for the base model. 

Run the following command in terminal to serve the base model using vLLM

- The configuration used to serve the compressed model and the base model (in the [Base.ipynb](../02_Base_Performance_Benchmarking/Base.ipynb) notebook) is the same other than the model name and port.
- Make sure to run this command from the `05_Compressed_Performance_Benchmarking` directory

  
```bash
vllm serve \
  "../Llama_3.1_8B_Instruct_int8_dynamic" \
  --host 127.0.0.1 \
  --port 8001 \
  --gpu-memory-utilization 0.6 \
  --tensor-parallel-size 1 \
  --pipeline-parallel-size 1 \
  --max-model-len 2048
```

Once the server starts, you will see something like this:

```INFO:     Started server process [166518]```\
```INFO:     Waiting for application startup.```\
```INFO:     Application startup complete.```

**NOTE** You may encounter the following warning when serving the model:
`The tokenizer you are loading from '../base_model' with an incorrect regex pattern... This will lead to incorrect tokenization.`

#### A test run to see if the vLLM server is accessible
We use a helper function **generate** (defined in [utils.py](./utils.py)) to simplify sending requests to our locally-served vLLM model.

This function wraps the OpenAI-compatible Chat Completions API exposed by vLLM.

In [ ]:
# For non streaming results
response = generate(
    model="../Llama_3.1_8B_Instruct_int8_dynamic",
    prompt="What is photosynthesis?",
    host="127.0.0.1",
    port=8001,
    api_key="empty",
    max_tokens=512,
)
print(response)

In [ ]:
# For streaming results
res = ""
for chunk in stream(
    model="../Llama_3.1_8B_Instruct_int8_dynamic",
    prompt="What is photosynthesis?",
    host="127.0.0.1",
    port=8001,
    api_key="empty",
    max_tokens=512,
):
    res += chunk
    print(chunk, end="", flush=True)

## Checking GPU vRAM
Loading the compressed model with the configuration defined in the above command will take approximately 28GB. It may seem surprising that a compressed 8.5 GB model consumes ~118 GB GPU memory. This is expected behavior in vLLM, due to how memory is allocated during inference. The main contributors are:

1. **Model Weights (~8.5 GB)**

    The size of your compressed model stored on disk (INT8, FP16, etc.). 
    Loaded once into GPU memory.
   
2. **Runtime GPU Memory (~6 GB)**

- vLLM reserves extra memory for:

- Parameter sharding

- CUDA kernels

- Attention buffers and temporary tensors

- Weight adapters and padded tensors

This adds ~4–8 GB depending on the model.

3. **KV Cache (~14 GB)**

- Stores key/value tensors for each generated token to avoid recomputation.

- Memory grows with sequence length, model hidden size, and concurrency.

- vLLM presets a large KV cache to support batching efficiently.


4. **GPU Memory Utilization Flag (--gpu-memory-utilization)**

``--gpu-memory-utilization`` is set to 0.6, meaning vLLM can utilize 60% of the total GPU memory. In this case, we have used one 46GB LS40 GPU, 60% of 46 is approx 28.

### Run Performance Benchmarking

Now that the **vLLM server for the compressed model** has been started, we can proceed with benchmarking its performance using **GuideLLM**.

Identify the following parameters:

- **target**: URL of the vLLM inference server started in the previous step  
  (e.g., `http://127.0.0.1:8001`)

- **output-path**: Path where benchmarking results will be saved

If needed, adjust the `target`, `output-path`, or benchmarking profile in the command below, then run it in a terminal.

**NOTES**:

- Ensure the vLLM server for the compressed model is running before executing the benchmark.
- If the vLLM server is running on a different port, update the `target` accordingly.
- Make sure you run the following command from the `05_Compressed_Performance_Benchmarking` directory.
- The same benchmarking configuration will be reused for evaluating the compressed model.

```bash
guidellm benchmark \
  --target "http://127.0.0.1:8001" \
  --profile sweep \
  --max-seconds 120 \
  --data "prompt_tokens=1024,output_tokens=512" \
  --output-path "../results/compressed_performance_benchmarks.json"
```

#### Results

The above command will result is multiple tables.

1. **Request Latency Statistics (Completed Requests)**

This table focuses on how **long** requests take and the latency characteristics of the server.

```text
ℹ Request Latency Statistics (Completed Requests)
|=============|=========|========|=========|=========|======|======|=======|=======|
| Benchmark   | Request Latency || TTFT             || ITL        || TPOT         ||
| Strategy    | Sec             || ms               || ms         || ms           ||
|             | Mdn     | p95    | Mdn     | p95     | Mdn  | p95  | Mdn   | p95   |
|-------------|---------|--------|---------|---------|------|------|-------|-------|
| synchronous | 7.6     | 7.9    | 87.9    | 445.7   | 14.7 | 14.7 | 14.8  | 15.5  |
| throughput  | 70.4    | 74.8   | 36149.3 | 40360.5 | 63.9 | 99.4 | 137.4 | 146.1 |
| constant    | 8.3     | 8.3    | 99.4    | 108.0   | 16.1 | 16.1 | 16.2  | 16.3  |
| constant    | 8.9     | 8.9    | 99.1    | 107.2   | 17.2 | 17.3 | 17.4  | 17.4  |
| constant    | 9.7     | 9.8    | 104.4   | 113.0   | 18.8 | 18.9 | 19.0  | 19.1  |
| constant    | 10.5    | 10.6   | 104.9   | 114.6   | 20.4 | 20.5 | 20.6  | 20.6  |
| constant    | 11.7    | 11.8   | 106.9   | 118.1   | 22.7 | 22.8 | 22.8  | 23.0  |
| constant    | 12.7    | 12.8   | 108.3   | 119.3   | 24.7 | 24.8 | 24.9  | 24.9  |
| constant    | 16.0    | 18.5   | 121.6   | 959.9   | 31.1 | 34.7 | 31.3  | 36.1  |
| constant    | 17.8    | 18.1   | 119.7   | 136.0   | 34.5 | 35.2 | 34.7  | 35.4  |
|=============|=========|========|=========|=========|======|======|=======|=======|

```

2.  **Server Throughput Statistics**

This table focuses on how many requests a server can handle per second. Throughput can be thought of as the **rate** (or time required) of processing. 
```text
Server Throughput Statistics
|=============|=====|======|=======|=======|========|========|=======|========|=======|========|
| Benchmark   | Requests                |||| Input Tokens   || Output Tokens || Total Tokens  ||
| Strategy    | Per Sec   || Concurrency  || Per Sec        || Per Sec       || Per Sec       ||
|             | Mdn | Mean | Mdn   | Mean  | Mdn    | Mean   | Mdn   | Mean   | Mdn   | Mean   |
|-------------|-----|------|-------|-------|--------|--------|-------|--------|-------|--------|
| synchronous | 0.1 | 0.1  | 1.0   | 1.0   | 139.6  | 148.9  | 68.2  | 67.6   | 68.2  | 207.3  |
| throughput  | 0.6 | 2.6  | 194.0 | 152.8 | 123.1  | 4262.7 | 966.7 | 1369.8 | 971.8 | 4200.5 |
| constant    | 0.4 | 0.4  | 4.0   | 3.3   | 456.2  | 465.9  | 217.6 | 209.7  | 217.8 | 643.0  |
| constant    | 0.7 | 0.7  | 6.0   | 6.1   | 779.7  | 789.8  | 326.7 | 353.9  | 327.1 | 1085.1 |
| constant    | 1.0 | 1.0  | 10.0  | 9.3   | 1103.9 | 1113.8 | 422.1 | 495.3  | 422.3 | 1518.7 |
| constant    | 1.3 | 1.2  | 14.0  | 12.8  | 1426.6 | 1437.8 | 498.8 | 634.7  | 499.5 | 1946.3 |
| constant    | 1.7 | 1.5  | 19.0  | 17.3  | 1753.6 | 1761.9 | 629.9 | 770.0  | 630.6 | 2361.2 |
| constant    | 2.0 | 1.7  | 25.0  | 22.0  | 2078.6 | 2085.8 | 746.2 | 901.6  | 747.0 | 2764.8 |
| constant    | 2.3 | 2.0  | 36.0  | 32.4  | 2401.2 | 2674.7 | 783.9 | 1110.9 | 786.0 | 3406.4 |
| constant    | 2.5 | 2.2  | 44.0  | 37.5  | 2733.0 | 2733.5 | 829.7 | 1123.5 | 831.5 | 3445.2 |
|=============|=====|======|=======|=======|========|========|=======|========|=======|========|


```
#### Compressed Model Performance Summary
1. Max concurrency under load: 44.0 (Concurrency Mdn)
2. Max output tokens per second under load: 829.7 (Output tokens per sec Mdn)
3. Request latency under load: 17.8 (Request Latency in secs Mdn)
4. Time to first token under load: 119.7 (TTFT ms Mdn)
5. Inter token latency under load: 34.5 (ITL ms Mdn)


#### SLO Analysis

Assume the Service Level Objective (SLO) is:

    TTFT ≤ 200 milliseconds for 95% of requests (p95) with optimal concurrency

At the highest tested concurrency of **44 requests**, the compressed model achieves a **p95 TTFT of 136.0 ms**, which comfortably satisfies the SLO.

This configuration meets the TTFT SLO of 200 ms for 95% of requests. Increasing concurrency beyond this point may push p95 TTFT above the SLO threshold and should be evaluated carefully in production scenarios.

For a workload of **1024 input tokens and 512 output tokens**, the system can sustain approximately **37–44 concurrent requests** while remaining within the TTFT ≤ 200 ms SLO. Reducing input and output token lengths (e.g., 512/256) allows the system to support more concurrent requests while maintaining compliance with the SLO.

#### Comparison with Base Model Performance

| Metric | Base Model | Compressed Model |
|------|-----------|------------------|
| p95 TTFT (ms) | 162.4 ms | **136.0 ms** |
| Max concurrency under SLO | 34 requests | **44 requests** |
| SLO satisfied | Yes | Yes |


In [ ]:
# Run this cell after the benchmarking process in the terminal completes
report = GenerativeBenchmarksReport.load_file(
    path="../results/compressed_performance_benchmarks.json",
)
compressed_benchmarks = report.benchmarks

In [ ]:
compressed_benchmarks[0]

GenerativeBenchmark(type_='generative_benchmark', config=BenchmarkConfig(id_='63d5d739-46cc-4fae-a870-8f0e98cdd6ae', run_id='c880f932-9fd7-4511-b0ef-21e8e794a9ad', run_index=0, strategy=SynchronousStrategy(type_='synchronous', worker_count=1, max_concurrency=1), constraints={'max_seconds': {'type_': 'max_duration', 'max_duration': 120.0, 'current_index': 0}}, sample_requests=10, warmup=TransientPhaseConfig(percent=None, value=None, mode='prefer_duration'), cooldown=TransientPhaseConfig(percent=None, value=None, mode='prefer_duration'), prefer_response_metrics=True, profile=SweepProfile(type_='sweep', completed_strategies=[SynchronousStrategy(type_='synchronous', worker_count=1, max_concurrency=1), ThroughputStrategy(type_='throughput', worker_count=10, max_concurrency=512, rampup_duration=0.0), AsyncConstantStrategy(type_='constant', worker_count=10, max_concurrency=512, rate=0.4312499999999999), AsyncConstantStrategy(type_='constant', worker_count=10, max_concurrency=512, rate=0.73749